## Summary

I first intended to make use of database. However, as I proceed problem occurs. 

It's very slow to write 5GB data into sqlite.

Meanwhile, reading data from mongodb is extremely slow.

Now I decide to use csv files instead. 

***This script is deprecated for the moment being***

## write csvs to sql

## toy example (illustation)

In [24]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('test_database')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS products (product_name text, price number)')
conn.commit()

data = {'product_name': ['Computer','Tablet','Monitor','Printer'],
        'price': [900,300,450,150]
        }

df = pd.DataFrame(data, columns= ['product_name','price'])
df.to_sql('products', conn, if_exists='replace', index = False)


c.execute('''  
SELECT * FROM products
          ''')

for row in c.fetchall():
    print (row)

('Computer', 900)
('Tablet', 300)
('Monitor', 450)
('Printer', 150)


## connect to database

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(database='aqf_database')
c = conn.cursor()

### read from csv and write to sql

In [4]:
sp500_op_ret = pd.read_csv("data/sp500_op_ret.csv")

In [8]:
c.execute(f'CREATE TABLE IF NOT EXISTS sp500_op_ret ({", ".join(list(sp500_op_ret.columns))})')
conn.commit()

sp500_op_ret.to_sql('sp500_op_ret', conn, if_exists='replace', 
                    index = False, chunksize=10000)

In [16]:
mapping_table = pd.read_csv("data/mapping_table.csv")

In [17]:
c.execute(f'CREATE TABLE IF NOT EXISTS mapping_table ({", ".join(list(mapping_table.columns))})')
conn.commit()

mapping_table.to_sql('mapping_table', conn, if_exists='replace', 
                    index = False, chunksize=10000)

## write in chunks 
(roughly 12 mins)

In [9]:
signed_predictors_dl_wide = pd.read_csv("data/signed_predictors_dl_wide.csv")

In [10]:
c.execute(f'CREATE TABLE IF NOT EXISTS signed_predictors_dl_wide ({", ".join(list(signed_predictors_dl_wide.columns))})')
conn.commit()

signed_predictors_dl_wide.to_sql('signed_predictors_dl_wide', conn, if_exists='replace', 
                                index = False, chunksize=10000)

## Retrieve data from database

In [33]:
c.execute('''DROP TABLE sp500_op_ret_permno''')

query = c.execute(''' 
        CREATE TABLE sp500_op_ret_permno
        AS 
            SELECT 
                op.secid as secid, op.date as date, op.exdate as exdate, op.cp_flag as cp_flag, op.strike_price as strike_price, op.best_bid as best_bid, op.best_offer as best_offer, op.volume as volume, op.open_interest as open_interest, op.impl_volatility as impl_volatility, op.delta as delta, op.gamma as gamma, op.vega as vega, op.theta as theta, op.optionid as optionid, op.cfadj as cfadj, op.days_no_trading as days_no_trading, op.days_to_exp as days_to_exp, op.forwardprice as forwardprice, op.spotprice as spotprice, op.adj_spot as adj_spot, op.ir_rate as ir_rate, op.mid_price as mid_price, op.option_ret as option_ret,
                mapping_table.permno as permno
            FROM sp500_op_ret AS op
            LEFT JOIN mapping_table 
            ON op.secid = mapping_table.secid
          ''')

In [ ]:
query = c.execute('''SELECT * from sp500_op_ret_permno''')
", ".join(list(map(lambda x: f"op.{x[0]} as {x[0]}" , query.description)))

In [41]:
query = c.execute('''
    SELECT op.secid
    FROM sp500_op_ret_permno AS op
    LEFT JOIN signed_predictors_dl_wide as pred
    ON op.permno = pred.permno
''')

In [4]:
sql = '''
    SELECT op.secid
    FROM sp500_op_ret_permno AS op
    LEFT JOIN signed_predictors_dl_wide as pred
    ON op.permno = pred.permno
'''
cnt = 0
for chunk in pd.read_sql_query(sql , conn, chunksize=5):
    print(chunk)
    cnt += 1
    if cnt > 5:
        break

   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015


In [2]:
?pd.DataFrame.from_records

Signature:
pd.DataFrame.from_records(
    data,
    index=None,
    exclude=None,
    columns=None,
    coerce_float=False,
    nrows=None,
) -> 'DataFrame'
Docstring:
Convert structured or record ndarray to DataFrame.

Creates a DataFrame object from a structured ndarray, sequence of
tuples or dicts, or DataFrame.

Parameters
----------
data : structured ndarray, sequence of tuples or dicts, or DataFrame
    Structured input data.
index : str, list of fields, array-like
    Field of array to use as the index, alternately a specific set of
    input labels to use.
exclude : sequence, default None
    Columns or fields to exclude.
columns : sequence, default None
    Column names to use. If the passed data do not have names
    associated with them, this argument provides names for the
    columns. Otherwise this argument indicates the order of the columns
    in the result (any names not found in the data will become all-NA
    columns).
coerce_float : bool, default False
    Attempt t

In [ ]:
pd.DataFrame.from_records(data=query.fetchall())

In [38]:
cols = 
# df = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)

['secid',
 'date',
 'exdate',
 'cp_flag',
 'strike_price',
 'best_bid',
 'best_offer',
 'volume',
 'open_interest',
 'impl_volatility',
 'delta',
 'gamma',
 'vega',
 'theta',
 'optionid',
 'cfadj',
 'days_no_trading',
 'days_to_exp',
 'forwardprice',
 'spotprice',
 'adj_spot',
 'ir_rate',
 'mid_price',
 'option_ret',
 'permno']

In [30]:
query.description

In [7]:
c.execute('''
SELECT name from sqlite_master where type= "table"
''')
c.fetchall()

[('sp500_op_ret',), ('signed_predictors_dl_wide',), ('mapping_table',)]

In [31]:
c.execute('''
SELECT * from sp500_op_ret
''')
", ".join(
    list(map(lambda x: x[0], c.description))
)

'secid, date, exdate, cp_flag, strike_price, best_bid, best_offer, volume, open_interest, impl_volatility, delta, gamma, vega, theta, optionid, cfadj, days_no_trading, days_to_exp, forwardprice, spotprice, adj_spot, ir_rate, mid_price, option_ret'

In [13]:
df

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
0,5015,1996-01-31 00:00:00,1996-07-20 00:00:00,C,20.0,4.625,5.000,15,25,0.380062,...,10980421,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367
1,5048,1996-01-31 00:00:00,1996-08-17 00:00:00,P,40.0,3.000,3.375,5,37,0.245716,...,11613022,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347
2,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,C,65.0,5.875,6.375,4,420,0.167298,...,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721
3,5049,1996-01-31 00:00:00,1996-08-17 00:00:00,C,70.0,3.250,3.625,13,3806,0.149214,...,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186
4,5061,1996-01-31 00:00:00,1996-06-22 00:00:00,C,30.0,3.375,3.750,5,254,0.324523,...,10914516,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825526,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,C,13.0,1.150,1.500,134,665,0.437066,...,143779767,1,0,227,12.112223,12.310,12.3100,0.002822,1.3250,-0.068384
3825527,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,C,15.0,0.100,0.750,5,14,0.335686,...,143779769,1,0,227,12.112223,12.310,12.3100,0.002822,0.4250,0.055073
3825528,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,P,11.0,0.200,1.100,3,111,0.308935,...,143779782,1,0,227,12.112223,12.310,12.3100,0.002822,0.6500,0.020419
3825529,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,P,12.0,1.100,1.600,53,35,0.372075,...,143779783,1,0,227,12.112223,12.310,12.3100,0.002822,1.3500,-0.016824


## Use MongoDB

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
data = pd.read_csv("data/signed_predictors_dl_wide.csv") # nrows=100000)

In [5]:
data.shape

(4890049, 204)

In [51]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["aqf_database"]
mycol = mydb["signed_predictors_dl_wide"]

destroy existed collection

In [53]:
mycol.delete_many({})

insert 10000 rows costs around 3 seconds



so it takes around 500x3 = 1500 seconds, or around half an hour

In [54]:
chunk_size= 10000
for i in range(data.shape[0]//chunk_size + 1):
    if i % 10 == 0:
        print(f"...Have written {i * chunk_size} rows...")
    # takes around 3 seconds for per insert 
    mycol.insert_many(
        data.iloc[i*chunk_size:(i+1) * chunk_size].to_dict("records")
    ) 
    

...Have written 0 rows...
...Have written 100000 rows...
...Have written 200000 rows...
...Have written 300000 rows...
...Have written 400000 rows...
...Have written 500000 rows...
...Have written 600000 rows...
...Have written 700000 rows...
...Have written 800000 rows...
...Have written 900000 rows...
...Have written 1000000 rows...
...Have written 1100000 rows...
...Have written 1200000 rows...
...Have written 1300000 rows...
...Have written 1400000 rows...
...Have written 1500000 rows...
...Have written 1600000 rows...
...Have written 1700000 rows...
...Have written 1800000 rows...
...Have written 1900000 rows...
...Have written 2000000 rows...
...Have written 2100000 rows...
...Have written 2200000 rows...
...Have written 2300000 rows...
...Have written 2400000 rows...
...Have written 2500000 rows...
...Have written 2600000 rows...
...Have written 2700000 rows...
...Have written 2800000 rows...
...Have written 2900000 rows...
...Have written 3000000 rows...
...Have written 3100000